In [ ]:
import torch
from torch import nn, optim
import cv2
import numpy as np
import torch.fft
import time
import matplotlib.pyplot as plt
import math
from skimage import io
import os
import scipy.io as scio
%matplotlib inline
from PIL import Image
from model import Net

magnification_factor=1.3e5
image_size = 64

t0=time.process_time()
print(torch.cuda.is_available())

Define the signal to be transmitted

In [ ]:
#lr=0.0001
lr=0.01
model = Net(n_channels=1, n_classes=1, bilinear=False)
criterion = nn.MSELoss()

if torch.cuda.is_available():
    model.cuda()

optvars = [{'params': model.parameters()}]
optimizier = torch.optim.Adam(optvars, lr=lr)

path='./images/DIV_0810.png'

img = cv2.imread(path)
img2 = cv2.resize(img, (image_size,image_size))
gray = cv2.split(img2)[2]

gray=gray/255
row=30

# obtain the original signal 
#gray=gray[row,:]
gray=gray/np.max(gray)

filename,_=os.path.splitext(os.path.basename(path))
scio.savemat(('./image_mat/{}.mat'.format(filename)),{'data':gray})
gray_res=gray.flatten()
print(gray)

gray_res=torch.from_numpy(gray_res)
gray_res_cuda=gray_res.cuda()
gray_res_cuda = gray_res_cuda.double()

plt.figure(figsize=(10,10))
plt.imshow(gray,cmap='gray')

signal=[]
for i in range(65):
    if i>0:
        signal.append(i)

In [ ]:
def ts_to_np(ts):
    
    return ts.detach().cpu().squeeze().numpy()
def np_to_ts(np):
    return torch.tensor(np).unsqueeze(0)


In [ ]:
def plot_loss_save(k,iters,y_loss,loss):
    #plt.cla()
    iters.append(k)
    loss_np=ts_to_np(loss)
    y_loss.append(loss_np)                 
      

def plot_loss(iters,y_loss):
    plt.plot(iters, y_loss, label='Loss rate', color='g')
    plt.xlabel('Iteration')
    plt.ylabel('MSE')
    plt.title('loss')
    plt.legend()   
    plt.pause(1.0)

The fixed input of the UNN

In [ ]:
#Input of unn
img_amp=torch.zeros(1,1,512,512)
input_random = img_amp.uniform_()*0.1
net_input =input_random.cuda() 
net_input_saved = net_input.detach().clone()

net_input_np=ts_to_np(net_input)
plt.figure()
plt.imshow(net_input_np,cmap='gray')
print(net_input)

In [ ]:
# add gaussian noise in the pattern due to the experimental consideration
gaussian_rand=np.random.random((512,512))
gaussian_rand=gaussian_rand-np.mean(gaussian_rand)
plt.figure()
plt.imshow(gaussian_rand,cmap='gray')
print(np.sum(gaussian_rand))

Generate a series of random amplitude-only patterns with UNN

In [ ]:
#generated measurements by unn: result
result=[]
ii=1

#pattern directory
os.makedirs('./pattern_13_{}/'.format(filename),exist_ok=True)
path_pattern=os.path.join('./pattern_13_{}/'.format(filename))

#loop to generate a series of random amplitude-only patterns
for i in range(image_size*image_size):
    iters=[]
    y_loss=[]
    
    model =  Net(n_channels=1, n_classes=1, bilinear=False)
    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        model.cuda()

    optvars = [{'params': model.parameters()}]
    optimizier = torch.optim.Adam(optvars, lr=lr)
    loss_item=1
    k=0
    while(loss_item>1e-15):
        output = model(net_input)

        output = torch.squeeze(output)
        
        #physical model
        recons = torch.fft.fftn(output)
        final = recons
        final = final.double()
        #print(final)
        final_spi = final[0][0]/magnification_factor
        #print(final_spi)
        
        loss = criterion(final_spi,gray_res_cuda[i])
        loss_item=ts_to_np(loss)
        plot_loss_save(k,iters,y_loss,loss)
        optimizier.zero_grad()

        loss.backward()
        optimizier.step()

        print ('LOOP %02d, Iteration %05d, loss %.20f ' % (i,k, loss.item()), '\r', end='')
        k=k+1
        if k>2000:
            break
        
    final_np=final_spi.detach().cpu().numpy()
    result.append(final_np)
    
    output=output.detach().cpu().numpy()
         
    output = (output+gaussian_rand)/2
    output_1 = (1+output)/2
    output_2 = (1-output)/2
    
    numm_1=str(ii).rjust(5,'0')
    numm_2=str(ii+1).rjust(5,'0')
    path_1=path_pattern+numm_1+'.png'
    path_2=path_pattern+numm_2+'.png'
    io.imsave(path_1,output_1)
    io.imsave(path_2,output_2)
    
    ii=ii+2

In [ ]:
print("time:",time.process_time()-t0)

result=np.array(result)

print(result)

In [ ]:
result_norm=result/np.max(result)
result_norm=result_norm.reshape((image_size,image_size))

Calculate the PSNR and SSIM

In [ ]:
import numpy as np

mse_value = np.mean((result_norm-gray)**2)
print("mse_value:",mse_value)
#mse_value=total_loss.item()
if mse_value<1.0e-10:
    PSNR= 100
PIXEL_MAX=np.max(result_norm)
PSNR=20*math.log10(1/math.sqrt(mse_value))
print("PSNR:",PSNR)

Test the quality of the signal encoded into the generated random pattern

In [ ]:
all_pixel_list=[]
k=1

rand_1=np.random.random((512,512))

rand_2=np.random.random((512,512))
for i in range(image_size*image_size):
    patternpath1 = path_pattern+str(k).rjust(5,'0')+'.png'
    patternpath2 = path_pattern+str(k+1).rjust(5,'0')+'.png'
    img_1=Image.open(patternpath1)
    img_1=np.array(img_1)
    img_2=Image.open(patternpath2)
    img_2=np.array(img_2)
    all_pixel=abs(np.sum(img_1)-np.sum(img_2))/magnification_factor
    
    all_pixel_list.append(all_pixel/255)
    
    k=k+2


In [ ]:
res=np.array(all_pixel_list)
res=res/np.max(res)
res=res.reshape((image_size,image_size))
print(res)
mse_value = np.mean((res-gray)**2)
print("mse_value:",mse_value)
#mse_value=total_loss.item()
if mse_value<1.0e-10:
    PSNR= 100
PIXEL_MAX=np.max(res)
PSNR=20*math.log10(1/math.sqrt(mse_value))
print("PSNR:",PSNR)